In [14]:
import tensorflow as tf
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
from tensorflow.keras import layers, models

In [16]:
input_shape = 15000 # Replace None with the fixed sequence length if known

def unet_1d(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    # Encoder
    conv1 = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(inputs)
    conv1 = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)
    
    conv2 = layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(pool1)
    conv2 = layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(conv2)
    pool2 = layers.MaxPooling1D(pool_size=2)(conv2)
    
    conv3 = layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(pool2)
    conv3 = layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(conv3)
    pool3 = layers.MaxPooling1D(pool_size=2)(conv3)
    
    # Bottleneck
    conv4 = layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(pool3)
    conv4 = layers.Conv1D(512, kernel_size=3, padding="same", activation="relu")(conv4)
    
    # Decoder
    up5 = layers.UpSampling1D(size=2)(conv4)
    concat5 = layers.Concatenate()([up5, conv3])
    conv5 = layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(concat5)
    conv5 = layers.Conv1D(256, kernel_size=3, padding="same", activation="relu")(conv5)
    
    up6 = layers.UpSampling1D(size=2)(conv5)
    concat6 = layers.Concatenate()([up6, conv2])
    conv6 = layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(concat6)
    conv6 = layers.Conv1D(128, kernel_size=3, padding="same", activation="relu")(conv6)
    
    up7 = layers.UpSampling1D(size=2)(conv6)
    concat7 = layers.Concatenate()([up7, conv1])
    conv7 = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(concat7)
    conv7 = layers.Conv1D(64, kernel_size=3, padding="same", activation="relu")(conv7)
    
    # Output
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation="softmax")(conv7)
    
    model = models.Model(inputs, outputs)
    return model

# Model Parameters
input_shape = (input_shape, 3)  
num_classes = 7
model = unet_1d(input_shape, num_classes)

# Model Summary
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 15000, 3)]           0         []                            
                                                                                                  
 conv1d_100 (Conv1D)         (None, 15000, 64)            640       ['input_8[0][0]']             
                                                                                                  
 conv1d_101 (Conv1D)         (None, 15000, 64)            12352     ['conv1d_100[0][0]']          
                                                                                                  
 max_pooling1d_22 (MaxPooli  (None, 7500, 64)             0         ['conv1d_101[0][0]']          
 ng1D)                                                                                      